### Imports and constants

In [ ]:
import spacy
import re
import datetime

import pickle


# =================== CONSTANTS ===================
SPACE = ' '
COLON = ':'
DASH = '-'
DOT = '.'

UNIT_THRESHOLD = 5
TIME_THRESHOLD = 600

LANGUAGE = 'es'

### UNIT CLASS

In [ ]:
import datetime

class Unit:
    def __init__(self, dt, sender, message):
        self.dt = self.get_datetime(dt)
        self.sender = sender
        self.message = message
    
    def same_sender(self, sender):
        return self.sender == sender
    
    def add_text(self, text):
        self.message += '. '.join([text])
    
    def set_dt(self, dt):
        self.dt = dt
    
    def timedelta(self, dt):
        if self.dt > dt:
            return (self.dt-dt).seconds
        return (dt-self.dt).seconds

    def get_datetime(self, dt):
        date, time = dt.split(',')
        month, day, year = date.split('/')
        hour, minutes = time.split(':')
        year = 2000 + int(year)
        return datetime.datetime(year, int(month), int(day), int(hour), int(minutes))

### GET_CORPUS AND PRE_PROCESSING

In [ ]:
reg = [
    [r'(\w)\1{2,}',r'\1'],
    [r'qu\w',''],
    [r'no\w','no'],
    ["jaj[ja]+",""],
    ["<media omitted>", ""],
    ["mñna", "mañana"],
    ["cba", "Cordoba"],
    ["si[si]+", "si"],
    ["hno", "hermano"],
    ["hdp", "insulto"],
    ["dpto", "departamento"]
]
STOP_WORDS = {"a","y", "o", "dale", "hola", \
              "ok", "oka", "ver", "estar", \
              "decir", "pasar", "che"}

In [ ]:
def get_corpus(corpus_filepath):
    # Get text.
    with open(corpus_filepath) as f:
        lines = f.read().lower()
    lines = lines.split('\n')
    corpus = []
    # Get list of Units.
    # Format of lines: date, time - sender: message
    for line in lines:
        unit = line.split(DASH)
        if len(unit) > 1:
            for exp in reg:
                unit[1] = re.sub(exp[0],exp[1],unit[1])
            if len(unit[0].split(',')) > 1 and len(unit[1].split(COLON)) > 1:
                unit = Unit(unit[0],unit[1].split(COLON)[0], unit[1].split(COLON)[1])
                # Messages with no less than UNIT_THRESHOLD
                ''' TODO: Make experimentation with this
                    if len(unit.message.split(SPACE)) < UNIT_THRESHOLD:
                    continue
                '''
                if len(corpus) > 0 and corpus[-1].same_sender(unit.sender) and \
                    corpus[-1].timedelta(unit.dt) < TIME_THRESHOLD:
                        corpus[-1].add_text(unit.message)
                        corpus[-1].set_dt(unit.dt)
                else:
                    corpus.append(unit)
    return corpus

In [ ]:
def pre_processing(corpus_filepath):
    corpus = get_corpus(corpus_filepath)
    # Skip units which has less than UNIT_THRESHOLD words.
    senders = []
    messages = []
    original_messages = []
    nlp = spacy.load(LANGUAGE)
    nlp.Defaults.stop_words|= STOP_WORDS
    for unit in corpus:
        senders.append(unit.sender)
        message = nlp(unit.message)
        # Remove non alpha words and make words lower with lemmatization.
        words = [word.lemma_.lower() for word in message if word.is_alpha]
        # Remove stop words.
        message = nlp(' '.join(words))
        words = [word.text for word in message if not word.is_stop]
        # Remove words with less than 3 characters.
        words = [word for word in words if len(word.strip()) > 2]
        if len(words) < UNIT_THRESHOLD:
            continue
        original_messages.append(unit.message)
        messages.append(words)
    return messages, original_messages